In [24]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
'''
Tried these features not much difference: play around with them if you want. 



df['log_popularity'] = np.log1p(df['popularity'])
test['log_popularity'] = np.log1p(test['popularity'])
df['log_budget'] = np.log1p(df['budget'])

df['ratio_budget_runtime'] = (df['log_budget'] / df['runtime'])
df['ratio_budget_popularity'] = df['log_budget'] / df['log_popularity']
df['ratio_budget_year'] = df['log_budget'] / df['release_year'] #
df['budget_to_mean_year'] = df['log_budget'] / df.groupby('release_year')['log_budget'].transform('mean')
df['ratio_popularity_year'] = df['log_popularity'] / df['release_year']
df['popularity_to_mean_year'] = df['log_popularity'] / df.groupby('release_year')['log_popularity'].transform('mean')

df['ratio_budget_year'].fillna(df['ratio_budget_year'].mean(),inplace=True)
df['budget_to_mean_year'].fillna(df['budget_to_mean_year'].mean(),inplace=True) #fill na with test vals?
df['ratio_budget_runtime'].fillna(df['ratio_budget_runtime'].mean(),inplace=True) #fill na with test vals?

le = LabelEncoder()
df['prodcomp']=le.fit_transform(df['production_companies'])
test['prodcomp']=le.fit_transform(test['production_companies'])

gender of cast feature might help
'''

In [25]:
df = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2train.csv", header=0) 
df.head()
test = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2test.csv")

In [26]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [27]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [28]:
df['revenue'] = np.log1p(df['revenue'])

In [29]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [30]:
missing_values_table(df)
# we have a good amount of missing variables that have to be either dropped or filled as it is a kaggle competition and we cannot 
# drop rows on the test set

Your selected dataframe has 24 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
belongs_to_collection,2396,79.9
homepage,2054,68.5
tagline,597,19.9
Keywords,276,9.2
production_companies,156,5.2
production_countries,55,1.8
spoken_languages,20,0.7
crew,16,0.5
cast,13,0.4
overview,8,0.3


In [35]:
df['isbelongs_to_collectionNA'] = 0
df.loc[df['belongs_to_collection'].isnull(), 'isbelongs_to_collectionNA'] = 1

test['isbelongs_to_collectionNA'] = 0
test.loc[test['belongs_to_collection'].isnull(), 'isbelongs_to_collectionNA'] = 1

In [36]:
df['isTaglineNA'] = 0
df.loc[df['tagline'].isnull(), 'isTaglineNA'] = 1
test['isTaglineNA'] = 0
test.loc[test['tagline'].isnull(), 'isTaglineNA'] = 1

In [37]:
#not used
df = df.drop(["homepage","belongs_to_collection", "poster_path", 'tagline','Keywords'], axis = 1)
test = test.drop(["homepage","belongs_to_collection", "poster_path",'tagline','Keywords'], axis = 1)
test['status'].fillna('Released',inplace=True)

In [38]:
le = LabelEncoder()
df['status']=le.fit_transform(df['status'])
test['status']=le.fit_transform(test['status'])


In [39]:
test['status']=le.fit_transform(test['status'])

In [41]:
df['original_language']=le.fit_transform(df['original_language'])
test['original_language']=le.fit_transform(test['original_language'])

In [45]:
df['cast'].fillna('0',inplace=True)
df['crew'].fillna('0',inplace=True)

test['cast'].fillna('0',inplace=True)
test['crew'].fillna('0',inplace=True)
# since we count crew and cast alter it is fine to just fill with as we it wont affect the feature

In [46]:
print (df['genres'].mode()[0])
print (df['spoken_languages'].mode()[0])
print (df['production_companies'].mode()[0])
print (df['production_countries'].mode()[0])

[{'id': 18, 'name': 'Drama'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'name': 'Paramount Pictures', 'id': 4}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]


In [47]:
df['genres'].fillna(df['genres'].mode()[0], inplace=True)
df['spoken_languages'].fillna(df['spoken_languages'].mode()[0], inplace=True)
df['production_companies'].fillna(df['production_companies'].mode()[0], inplace=True)
df['production_countries'].fillna(df['production_countries'].mode()[0], inplace=True)
#test
test['genres'].fillna(test['genres'].mode()[0], inplace=True)
test['spoken_languages'].fillna(test['spoken_languages'].mode()[0], inplace=True)
test['production_companies'].fillna(test['production_companies'].mode()[0], inplace=True)
test['production_countries'].fillna(test['production_countries'].mode()[0], inplace=True)
#print (df['spoken_languages'].mode()[0])
#print (df['production_companies'].mode()[0])
#print (df['production_countries'].mode()[0])

In [49]:
missing_values_table(df)

Your selected dataframe has 21 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
overview,8,0.3
runtime,2,0.1


In [50]:
df["overview"].fillna(0)
test["overview"].fillna(0)

0       Ash and friends (this time accompanied by newc...
1       When an abused wife grows to giant size becaus...
2       Good-natured astronomer Sam is devastated when...
3       A mother's last wishes send twins Jeanne and S...
4       In 1972, a seemingly typical shoestring budget...
                              ...                        
4393    A young man is plunged into a life of subterfu...
4394    Three friends, whose lives have been drifting ...
4395    Frank Galvin is a down-on-his luck lawyer, red...
4396    For 19-year-old Jay, fall should be about scho...
4397    Twelve episodic tales in the life of a Parisia...
Name: overview, Length: 4398, dtype: object

In [52]:
test['runtime'].fillna(test['runtime'].mean(),inplace=True)
df['runtime'].fillna(test['runtime'].mean(),inplace=True)

In [54]:
missing_values_table(test)

Your selected dataframe has 20 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
overview,14,0.3
title,3,0.1
release_date,1,0.0
year,1,0.0


In [55]:
print(test[test['release_date'].isnull()])
print(test[test['year'].isnull()])


       id  budget                         genres    imdb_id  \
828  3829       0  [{'id': 18, 'name': 'Drama'}]  tt0210130   

     original_language              original_title  \
828                  9  Jails, Hospitals & Hip-Hop   

                                              overview  popularity  \
828  Jails, Hospitals &amp; Hip-Hop is a cinematic ...    0.009057   

                          production_companies  \
828  [{'name': 'Paramount Pictures', 'id': 4}]   

                                  production_countries release_date  runtime  \
828  [{'iso_3166_1': 'US', 'name': 'United States o...          NaN     90.0   

                             spoken_languages  status  \
828  [{'iso_639_1': 'en', 'name': 'English'}]       1   

                          title cast crew  year  isbelongs_to_collectionNA  \
828  Jails, Hospitals & Hip-Hop   []    0   NaN                          1   

     isTaglineNA  
828            0  
       id  budget                         genres   

In [56]:
test['release_date'].fillna('3/20/01',inplace=True)

In [57]:
test['year'].fillna("2000", inplace = True)

In [58]:
test['title'].fillna(test['original_title'],inplace=True)

In [59]:
missing_values_table(test)

Your selected dataframe has 20 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
overview,14,0.3


In [64]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [65]:
Y = df["revenue"]

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   overview                   2992 non-null   object 
 7   popularity                 3000 non-null   float64
 8   production_companies       3000 non-null   object 
 9   production_countries       3000 non-null   object 
 10  release_date               3000 non-null   object 
 11  runtime                    3000 non-null   float64
 12  spoken_languages           3000 non-null   object 
 13  status                     3000 non-null   int32

In [67]:
Y.head()

0    16.326300
1    18.370959
2    16.387512
3    16.588099
4    15.182615
Name: revenue, dtype: float64

In [68]:
#df["IsCollection"].value_counts()

In [69]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,title,cast,crew,revenue,year,isbelongs_to_collectionNA,isTaglineNA
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,7,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",0,Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,0,0
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,7,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",0,The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,0,0
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,7,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",0,Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,1,0
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,13,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",0,Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012,1,1
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,18,마린보이,Marine Boy is the story of a former national s...,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",...,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",0,Marine Boy,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009,1,1


In [70]:
df["budget_year_ratio"] = df['budget'] / (df['year'] * df['year'])
test["budget_year_ratio"] = test['budget'] / (test['year'] * test['year'])

In [71]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,title,cast,crew,revenue,year,isbelongs_to_collectionNA,isTaglineNA,budget_year_ratio
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,7,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",0,Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",16.326300,2015,0,0,3.448085
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,7,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",0,The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",18.370959,2004,0,0,9.960120
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,7,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",0,Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",16.387512,2014,1,0,0.813570
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,13,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",0,Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16.588099,2012,1,1,0.296432
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,18,마린보이,Marine Boy is the story of a former national s...,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",...,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",0,Marine Boy,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",15.182615,2009,1,1,0.000000


In [72]:
#little function for for finding an overview
def extractOverviewInfo(df):
    df['has_overview'] = df['overview'].apply(lambda x: isinstance(x, str))
    df['overview_len'] = df['overview'].apply(lambda x: len(x) if isinstance(x, str) else 0)
    return df

In [73]:
df = extractOverviewInfo(df)


In [74]:
test = extractOverviewInfo(test)

In [75]:
df['has_overview'] = df['has_overview'].astype(int)
test['has_overview'] = test['has_overview'].astype(int)
df = df.drop(['overview'], axis = 1)
test = test.drop(['overview'], axis = 1)


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   popularity                 3000 non-null   float64
 7   production_companies       3000 non-null   object 
 8   production_countries       3000 non-null   object 
 9   release_date               3000 non-null   object 
 10  runtime                    3000 non-null   float64
 11  spoken_languages           3000 non-null   object 
 12  status                     3000 non-null   int32  
 13  title                      3000 non-null   objec

In [77]:
df['budget_runtime'] = (df['budget'] + 1)/(df['runtime']+1)
df['poularity_year'] = (df['popularity']+1)/(df['year']+1)
df['budget_popularity'] = (df['budget']+1)/(df['popularity']+1)
df['budget_year'] = (df['budget'] + 1)/(df['year']+1)
df['runtime_year'] = (df['runtime']+1)/(df['year']+1)

#test


In [78]:
test['budget_runtime'] = (test['budget'] + 1)/(test['runtime']+1)
test['poularity_year'] = (test['popularity']+1)/(test['year']+1)
test['budget_popularity'] = (test['budget']+1)/(test['popularity']+1)
test['budget_year'] = (test['budget'] + 1)/(test['year']+1)
test['runtime_year'] = (test['runtime']+1)/(test['year']+1)

In [79]:
# want to add 
# number of crew/cast
# Production companies 
# possibly add day of week, day of year, month, or quarter 
# number of keywords 
# has homepage 
# original language? 
# 

In [80]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,isbelongs_to_collectionNA,isTaglineNA,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,7,Hot Tub Time Machine 2,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,...,0,0,3.448085,1,155,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,7,The Princess Diaries 2: Royal Engagement,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,...,0,0,9.960120,1,393,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,7,Whiplash,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,...,1,0,0.813570,1,130,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,13,Kahaani,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,...,1,1,0.296432,1,581,9756.105691,0.002074,2.874298e+05,596.125683,0.061103
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,18,마린보이,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,...,1,1,0.000000,1,168,0.008403,0.001069,4.655342e-01,0.000498,0.059204


In [81]:
#X = df[["popularity", "runtime", "budget", "budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len' ]]

In [82]:
#df['revenue'] = np.log1p(df['revenue'])
y = df[["revenue"]]

In [83]:
members = df["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_cast"] = num_cast

In [84]:
members = df["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_crew"] = num_cast

In [85]:
members = test["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_cast"] = num_cast

In [86]:
members = test["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_crew"] = num_cast

In [87]:
test.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,num_cast,num_crew
0,3001,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",tt1226251,19,ディアルガVSパルキアVSダークライ,3.851534,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,...,0.000000,1,335,0.010989,0.002416,0.206120,0.000498,0.045319,7,2
1,3002,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",tt0051380,9,Attack of the 50 Foot Woman,3.559789,"[{'name': 'Woolner Brothers Pictures Inc.', 'i...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,...,0.020777,1,162,1333.348485,0.002215,19299.357931,42.739679,0.032054,10,8
2,3003,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",tt0118556,9,Addicted to Love,8.085194,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,...,0.000000,1,362,0.009901,0.004547,0.110069,0.000501,0.050551,9,10
3,3004,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",tt1255953,12,Incendies,8.596012,"[{'name': 'TS Productions', 'id': 313}, {'name...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,...,1.683127,1,302,51908.404580,0.004772,708627.813304,3381.402785,0.065142,23,69
4,3005,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",tt0418753,9,Inside Deep Throat,3.217680,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,...,0.497509,1,894,21505.387097,0.002103,474194.580907,997.009472,0.046361,4,14


In [88]:
df.head()

,id,budget,genres,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,...,budget_year_ratio,has_overview,overview_len,budget_runtime,poularity_year,budget_popularity,budget_year,runtime_year,num_cast,num_crew
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,7,Hot Tub Time Machine 2,6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,...,3.448085,1,155,148936.180851,0.003758,1.848089e+06,6944.444940,0.046627,24,72
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,7,The Princess Diaries 2: Royal Engagement,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,...,9.960120,1,393,350877.201754,0.004613,4.324841e+06,19950.125187,0.056858,20,9
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,7,Whiplash,64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,...,0.813570,1,130,31132.084906,0.032407,5.053601e+04,1637.717618,0.052605,51,64
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,13,Kahaani,3.174936,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,...,0.296432,1,581,9756.105691,0.002074,2.874298e+05,596.125683,0.061103,7,3
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,18,마린보이,1.148070,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,...,0.000000,1,168,0.008403,0.001069,4.655342e-01,0.000498,0.059204,4,2


In [89]:
# Genre 

genres = df["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
df["genres_list"] = genres_list


In [90]:

genresInMovie = df["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
df["isAction"] = action
df["isAdventure"] = adventure
df["isAnimation"] = animation
df["isComedy"] = comedy
df["isCrime"] = crime
df["isDocumentary"] = documentary
df["isDrama"] = drama
df["isFamily"] = family
df["isFantasy"] = fantasy
df["isForeign"] = foreign
df["isHorror"] = horror
df["isMusic"] = music
df["isMystery"] = mystery
df["isRomance"] = romance
df["isSciFi"] = scifi
df["isThriller"] = thriller


In [91]:
genres = test["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
test["genres_list"] = genres_list


In [92]:
genresInMovie = test["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
test["isAction"] = action
test["isAdventure"] = adventure
test["isAnimation"] = animation
test["isComedy"] = comedy
test["isCrime"] = crime
test["isDocumentary"] = documentary
test["isDrama"] = drama
test["isFamily"] = family
test["isFantasy"] = fantasy
test["isForeign"] = foreign
test["isHorror"] = horror
test["isMusic"] = music
test["isMystery"] = mystery
test["isRomance"] = romance
test["isSciFi"] = scifi
test["isThriller"] = thriller

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   popularity                 3000 non-null   float64
 7   production_companies       3000 non-null   object 
 8   production_countries       3000 non-null   object 
 9   release_date               3000 non-null   object 
 10  runtime                    3000 non-null   float64
 11  spoken_languages           3000 non-null   object 
 12  status                     3000 non-null   int32  
 13  title                      3000 non-null   objec

In [94]:
def date_features(df):
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['release_month'] = df['release_date'].dt.month
    df['release_day'] = df['release_date'].dt.day
    #df['release_quarter'] = df['release_date'].dt.quarter
    df['release_dayofweek'] = df['release_date'].dt.dayofweek
    df.drop(columns=['release_date'], inplace=True)
    return df

df = date_features(df)
test = date_features(test)

In [ ]:
def date_adjust(df):
    # getting the release date into proper format
    df[['release_month', 'release_day','release_year']] = df['release_date'].str.split('/',expand=True).replace(np.nan,0).astype(int)
    
    #df.loc[df['release_year'] <=18, 'release_year'] += 2000
    #df.loc[df['release_year'] > 18, 'release_year'] += 1900
    releaseDate = pd.to_datetime(df['release_date'])
    df['release_week'] = releaseDate.dt.week
    df['release_dayofweek'] = releaseDate.dt.dayofweek
    df['release_quarter'] = releaseDate.dt.quarter
    df['original_english'] = df['original_language'].apply(lambda x: 1 if x == 'en' else 0)
    df = df.drop(['collection_name','original_language','release_date'], axis=1)


date_adjust(df)
date_adjust(test)

In [ ]:
'''
Tried these features not much difference 
df['log_popularity'] = np.log1p(df['popularity'])
test['log_popularity'] = np.log1p(test['popularity'])
df['log_budget'] = np.log1p(df['budget'])

df['ratio_budget_runtime'] = (df['log_budget'] / df['runtime'])
df['ratio_budget_popularity'] = df['log_budget'] / df['log_popularity']
df['ratio_budget_year'] = df['log_budget'] / df['release_year'] #
df['budget_to_mean_year'] = df['log_budget'] / df.groupby('release_year')['log_budget'].transform('mean')
df['ratio_popularity_year'] = df['log_popularity'] / df['release_year']
df['popularity_to_mean_year'] = df['log_popularity'] / df.groupby('release_year')['log_popularity'].transform('mean')

df['ratio_budget_year'].fillna(df['ratio_budget_year'].mean(),inplace=True)
df['budget_to_mean_year'].fillna(df['budget_to_mean_year'].mean(),inplace=True) #fill na with test vals?
df['ratio_budget_runtime'].fillna(df['ratio_budget_runtime'].mean(),inplace=True) #fill na with test vals?

le = LabelEncoder()
df['prodcomp']=le.fit_transform(df['production_companies'])
test['prodcomp']=le.fit_transform(test['production_companies'])

gender of cast feature might help
'''

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   popularity                 3000 non-null   float64
 7   production_companies       3000 non-null   object 
 8   production_countries       3000 non-null   object 
 9   runtime                    3000 non-null   float64
 10  spoken_languages           3000 non-null   object 
 11  status                     3000 non-null   int32  
 12  title                      3000 non-null   object 
 13  cast                       3000 non-null   objec

In [97]:
X = df[["popularity", "runtime", "budget", "isbelongs_to_collectionNA", "isTaglineNA","budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller", 'release_month', 'release_day','release_dayofweek'  
        , "status", 'original_language']]
# "isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller"

In [98]:
X.head()

,popularity,runtime,budget,isbelongs_to_collectionNA,isTaglineNA,budget_year_ratio,year,budget_runtime,poularity_year,budget_popularity,...,isMusic,isMystery,isRomance,isSciFi,isThriller,release_month,release_day,release_dayofweek,status,original_language
0,6.575393,93.0,14000000,0,0,3.448085,2015,148936.180851,0.003758,1.848089e+06,...,0,0,0,0,0,2,20,4,0,7
1,8.248895,113.0,40000000,0,0,9.960120,2004,350877.201754,0.004613,4.324841e+06,...,0,0,1,0,0,8,6,4,0,7
2,64.299990,105.0,3300000,1,0,0.813570,2014,31132.084906,0.032407,5.053601e+04,...,0,0,0,0,0,10,10,4,0,7
3,3.174936,122.0,1200000,1,1,0.296432,2012,9756.105691,0.002074,2.874298e+05,...,0,0,0,0,1,3,9,4,0,13
4,1.148070,118.0,0,1,1,0.000000,2009,0.008403,0.001069,4.655342e-01,...,0,0,0,0,1,2,5,3,0,18


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

In [ ]:
print(y)

In [ ]:
#xgbr.fit(X_train, y_train)

In [119]:
m = RandomForestRegressor(n_estimators=60, random_state = 42, min_samples_leaf = 5,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(min_samples_leaf=5, n_estimators=60, n_jobs=-1,
                      oob_score=True, random_state=42)

In [102]:
def rmse(y_pred, y_true):
    return np.sqrt(mean_squared_error(y_pred, y_true))

def print_rf_score(model):
    print(f'Train R2:   {model.score(X_train, y_train)}')
    print(f'Test R2:   {model.score(X_test, y_test)}')
    print(f'Train RMSE: {rmse(model.predict(X_train), y_train)}')
    print(f'Test RMSE: {rmse(model.predict(X_test), y_test)}')

In [120]:
print_rf_score(m)

Train R2:   0.7710002981847681
Test R2:   0.5422207268164349
Train RMSE: 1.4729909716043725
Test RMSE: 2.0198803099460236


In [149]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [50,60,70,80,90,100,110,120,130,140,150,200,400, 500],
    'max_depth' : [5,6,7,8,9,10,11,15,20],
    'min_samples_leaf' : [4,5,6,7,8,9,10,15]
}
CV_rfc = GridSearchCV(estimator=m, param_grid=param_grid, cv= 5, verbose = 2, scoring = 'neg_mean_squared_error')
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_estimator_)
bestResults = pd.DataFrame(CV_rfc.cv_results_)


Fitting 5 folds for each of 1008 candidates, totalling 5040 fits
[CV] max_depth=5, min_samples_leaf=4, n_estimators=50 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] . max_depth=5, min_samples_leaf=4, n_estimators=50, total=   1.3s
[CV] max_depth=5, min_samples_leaf=4, n_estimators=50 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] . max_depth=5, min_samples_leaf=4, n_estimators=50, total=   0.2s
[CV] max_depth=5, min_samples_leaf=4, n_estimators=50 ................
[CV] . max_depth=5, min_samples_leaf=4, n_estimators=50, total=   0.2s

...

[CV]  max_depth=20, min_samples_leaf=15, n_estimators=500, total=   1.0s
[CV] max_depth=20, min_samples_leaf=15, n_estimators=500 .............


C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  max_depth=20, min_samples_leaf=15, n_estimators=500, total=   1.0s


[Parallel(n_jobs=1)]: Done 5040 out of 5040 | elapsed: 37.1min finished
C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=500,
                      n_jobs=-1, oob_score=True, random_state=42)


In [ ]:
#best RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=500,
                     # n_jobs=-1, oob_score=True, random_state=42)

In [152]:
m2 = RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=500,
                      n_jobs=-1, oob_score=True, random_state=42)
m2.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=1500,
                      n_jobs=-1, oob_score=True, random_state=42)

In [153]:
print_rf_score(m2)

Train R2:   0.6733065369040178
Test R2:   0.5354523036605963
Train RMSE: 1.7593524447801687
Test RMSE: 2.0347578301027736


In [239]:
xgb = XGBRegressor(
        n_estimators = 15000, 
        learning_rate = 0.001,
        max_depth = 6, 
        subsample = 0.5, #.3
        colsample_bytree = 0.6,
eta = .1)
xgb.fit(X_train, y_train)

[11:43:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.6, eta=0.1, learning_rate=0.001, max_depth=6,
             n_estimators=15000, subsample=0.5)

In [144]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [165]:
model = XGBRegressor()
param_grid = {
        'n_estimators': [10000],
        'min_child_weight': [1, 5, 10],
        #"subsample" : [.3, .6, 1.0],
        'gamma': [0.5, 1, 1.5, 2],
        "eta": [0.01, 0.1, 1],
        #'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5,6,7]
}

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 30.8min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 47.5min finished


[01:50:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2.166808397562417
{'eta': 0.01, 'gamma': 2, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 10000}


In [ ]:
# results 'eta': 0.01, 'gamma': 2, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 10000

In [ ]:
xgb_full_train = XGBRegressor(objective = 'reg:squarederror',
                                  eta = 0.01,
                                  max_depth = 6,
                                  min_child_weight = 3,
                                  subsample = 0.8,
                                  colsample_bytree = 0.7,
                                  eval_metric = 'rmse',
                                  seed = 1,
                                  n_estimators = 2800)

In [ ]:
xgb_full_train.fit(X_train, y_train)

In [101]:
print_rf_score(xgb_full_train)

NameError: name 'print_rf_score' is not defined

In [ ]:
\

In [240]:
print_rf_score(xgb)

Train R2:   0.9321526300707716
Test R2:   0.5489358313092216
Train RMSE: 0.8017687685065473
Test RMSE: 2.0050108986297834


In [154]:
predictdf= test[["popularity", "runtime", "budget", "isbelongs_to_collectionNA", "isTaglineNA","budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller", 'release_month', 'release_day','release_dayofweek'  
        , "status", 'original_language']]

In [241]:
# results 'eta': 0.01, 'gamma': 2, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 10000
xgb2 = XGBRegressor(
        min_child_weight = 10,
        max_depth = 7, 
        subsample = 0.3, #.3
        colsample_bytree = 0.8,
        gamma = 2,
        n_estimators = 10000,
        eta = 0.01)

In [243]:
xgb2.fit(X,y)

[11:46:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.8, eta=0.01, gamma=2, max_depth=7,
             min_child_weight=10, n_estimators=10000, subsample=0.3)

In [230]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators = 20, max_depth = 8, min_samples_leaf=5)
reg.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GradientBoostingRegressor(max_depth=8, min_samples_leaf=5, n_estimators=20)

In [231]:
print_rf_score(reg)

Train R2:   0.8172619473956309
Test R2:   0.5592302873110876
Train RMSE: 1.3158219375295739
Test RMSE: 1.9819990649300208


In [178]:
xgb2.fit(X_train,y_train)

[11:27:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.8, eta=0.01, gamma=2, max_depth=7,
             min_child_weight=4, n_estimators=5000, subsample=0.3)

In [179]:
print_rf_score(xgb2)

Train R2:   0.9848241790208975
Test R2:   0.45287884126179656
Train RMSE: 0.3791916137855673
Test RMSE: 2.2082046916291853


In [ ]:
predictdf

In [ ]:
predictarray = np.asarray(predictdf)

In [155]:

submission = m2.predict(predictdf)
submission


array([14.33370243, 13.81975666, 15.72712082, ..., 17.13700669,
       15.04955172, 14.81046501])

In [232]:
submissionreg = reg.predict(predictdf)

In [244]:
xgbsubmission = xgb2.predict(predictdf)
xgbsubmission

array([14.918745, 14.608081, 17.667183, ..., 17.557154, 16.246262,
       15.738557], dtype=float32)

In [245]:
subdf = pd.DataFrame(xgbsubmission)
subdf

,0
0,14.918745
1,14.608081
2,17.667183
3,19.167780
4,14.769491
...,...
4393,17.268450
4394,18.687269
4395,17.557154
4396,16.246262


In [246]:
subdf["id"] = test["id"]
subdf.columns = ['revenue', 'id']

In [247]:
subdf = subdf[['id', 'revenue']]

In [248]:
subdf['revenue'] = np.round(np.expm1(subdf["revenue"]))

In [249]:
subdf

,id,revenue
0,3001,3013898.0
1,3002,2209066.0
2,3003,47071720.0
3,3004,211086784.0
4,3005,2596022.0
...,...,...
4393,7394,31593116.0
4394,7395,130550304.0
4395,7396,42167220.0
4396,7397,11367415.0


In [250]:
subdf.to_csv("Submission10.csv")

In [ ]:
# month released 
# week released etc. 
# number of keywords 
# cast gender? 